In [24]:
import numpy as np
import pandas as pd

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
X=[]
Y=[]
lines=[]

data_path='/content/drive/My Drive/Self-driving-car-dataset/data.txt'

images_path='/content/drive/My Drive/Self-driving-car-dataset/images/'

with open(data_path) as file:
    lines=file.readlines()
    

for line in lines:
    w=line.split()
    X.append(w[0])
    Y.append(float(w[1]))

In [27]:
for i,x in enumerate(X):
    X[i]=images_path+X[i]
    
X[4]

'/content/drive/My Drive/Self-driving-car-dataset/images/4.jpg'

In [28]:
import math

for i,y in enumerate(Y):
    Y[i]=(y*math.pi)/180
    
print(Y[89])

0.024609142453120045


In [29]:
split=int(.8*len(X))
X_train=X[0:split]
X_val=X[split:]
Y_train=Y[0:split]
Y_val=Y[split:]

print("train:",len(X_train),len(Y_train))
print("val:",len(X_val),len(Y_val))

train: 36324 36324
val: 9082 9082


In [30]:
from sklearn.metrics import mean_squared_error

z=[0]*len(Y_val)
mean_squared_error(Y_val, z)

0.19089104348993743

In [31]:
import numpy as np
import cv2

def rotate_image(img, smoothed_angle, degrees):
    rows,cols,_ = img.shape
    if smoothed_angle!=degrees:
        smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    
    return dst, smoothed_angle

In [32]:
i=0
import os
wheel_path='/content/drive/My Drive/Self-driving-car-dataset/wheel-4.jpg'
wheel=cv2.imread(wheel_path)
smoothed_angle=0
smoothed_angle_actual=0

wheel.shape

(240, 240, 3)

In [ ]:
#model.save('m1.h5')
# from keras.models import load_model 
import tensorflow.compat.v1 as tf
model_path='/content/drive/My Drive/Self-driving-car-dataset/model.h5'
model = tf.keras.models.load_model(model_path)

In [ ]:

for x in X_train:
    img=cv2.imread(x)/255
    cv2.imshow('frame',img)
    
    rad=model.predict(np.array([img]))[0][0]
    deg=(rad*180)/(math.pi)
    
    w,smoothed_angle=rotate_image(wheel,smoothed_angle,deg)
    cv2.imshow('Prediction',w)
    
    deg2=(Y_train[i]*180)/(math.pi)
    
#     s="Prediction: "+str(deg)+" Actual: "+str(deg2)+"\n"
#     s=s.encode()
#     os.write(1,s)
    w2,smoothed_angle_actual=rotate_image(wheel,smoothed_angle_actual,deg2)
    cv2.imshow('Actual',w2)
    
    i+=1;
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

IndexError: list index out of range